### Install all dependencies before running this code
#### pip install opencv-python
#### pip install Cmake
#### pip install Dlib
#### pip install face-recognition
#### pip install itertools-s
#### pip install pyttsx3
#### Install visual studio 2019 community version

In [1]:
import cv2
import os
import face_recognition
import numpy as np
import pandas as pd
from scipy import stats
import itertools
import pyttsx3
import threading
import pyodbc
from tkinter import *
import sys
from PIL import ImageTk,Image
from datetime import datetime

In [2]:
face_known_encodings=[]
face_known_name = []
pred_names = []
face_known_encodings_flat=[]
engine = pyttsx3.init()
cam = cv2.VideoCapture(0)

## Video to encodings

### This function will open your webcam and captures your encodings

In [3]:
def video2encoding(vid_source,names):
    global faces
    global panel
    faces=[]
    cam = cv2.VideoCapture(vid_source)
    frame_no = 0
    know_encodings = []
    root.update()
    while True: 

        # reading from frame 
        ret,frame = cam.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        panel = Label(image=image)
        panel.image = image
        panel.place(x=300, y=270)
        if len(know_encodings) == 45:
            break
        #print(ret)
        if ret==True:
            #print (frame_no)
            face_locations = face_recognition.face_locations(frame)
            if len(face_locations)!= 0:
                face_encoding = face_recognition.face_encodings(frame,known_face_locations=face_locations)
                know_encodings.append(face_encoding)
            for face_locations in face_locations:
                cv2.rectangle(frame,(face_locations[3],face_locations[0]), (face_locations[1],face_locations[2]), (255, 0, 0), 2)     
            frame_no = frame_no+1
            #cv2.imshow('img', frame)
            cv2.waitKey(1)
        else:
            break

    # Release all space and windows once done
    
    cam.release() 
    cv2.destroyAllWindows() 
    faces.append([names,know_encodings])
    print("done")
    export()

## Preprocessing for exporting new dataset

### This function will save the encodings in new data frame

In [4]:
df= pd.DataFrame()
def encoding_df (faces):
   
    for j in range(0,len(faces)):
        test = faces[j][1]
        name = faces[j][0]
        #print(name)
        a=list(itertools.chain(*test))
        b=list(itertools.chain(*a))
        df[[name]]=pd.DataFrame(b)
    return(df)

## Adding a new face to CSV

### This function will open the existing dataframe and adds new encodings into a column

In [5]:
source = "D:\\P1\\encoding.csv"
def add_new_encoding(source,faces,emp):
    df=pd.read_csv(source)
    for j in range(0,len(faces)):
        test = faces[j][1]
        name = faces[j][0]
        #print(name)
        a=list(itertools.chain(*test))
        b=list(itertools.chain(*a))
        #print(b)
        df[[emp]]=pd.DataFrame(b)
    return(df)

## Processing the data to required format

### Converts the csv into a format that the model can understand

In [6]:
def csv2encodings(data): 
    names=data.columns.values
    lengths=[]
    encodings=[]
    known_encodings=[]
    known_names=[]
    for i in names:
        data_1=data[i]
        data_1 = data_1.dropna()
        data_2=data_1.to_list()
        encodings.append(data_2)
        lengths.append(len(data_2)/128)

    for j in range(0,len(encodings)):
        for i in range(0,round(len(encodings[j])/128) ):
            x=0
            y=128
            known_encodings.append(np.array(encodings[j][x:y]))
            x = (x+128)+1
            y= y+128#

    for i in range(0,len(lengths)):
        for j in range(0,round(lengths[i])):
            known_names.append(names[i])
    #print(lengths)
    #print(names)
    return(known_names,known_encodings)

## Compressed to 2 functions - FOR FRONT END

### Compares the encodings in the csv with current encodings from the camera and detects the person in the frame.

In [7]:
def start():
    data = pd.read_csv("D:\\Downloads\\encoding.csv")
    face_known_name,face_known_encodings_flat=csv2encodings(data)
    cap = cv2.VideoCapture(0)
    name = ""

    try:
        while True:

            # Read the frame
            _, img = cap.read()

            # Detect the faces

            face_locations = face_recognition.face_locations(img)
            if len(face_locations)!= 0:
                face_encoding = face_recognition.face_encodings(img,known_face_locations=face_locations)[0] 

            else:
                pred_names = []
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(img,"finding new face",(25,25) ,font, 1.0, (255, 0, 0), 1)

            #Draw the rectangle around each face

            for face_locations in face_locations:
                cv2.rectangle(img,(face_locations[3],face_locations[0]), (face_locations[1],face_locations[2]), (255, 0, 0), 2)

                face_recognize = face_recognition.face_distance(face_known_encodings_flat,face_encoding)
                face_recognize_test = face_recognize < 0.6
                true= sum(face_recognize_test)
                if true >= 30:
                    best_match_index = np.argmin(face_recognize)
                    pred_names.append(face_known_name[best_match_index])
                    name = "identifying ...."
                    if len(pred_names) == 5:
                        name = stats.mode(pred_names)
                        name = name.mode.item()
                        engine.say("Good morning,"+name)
                        engine.say("Have a great day!")
                        engine.runAndWait()
                        engine.stop()
                        pred_names = []

                elif true < 30:
                    name = "unknown face"
                    engine.say("Sorry !")
                    engine.say("Not able to identify who you are")
                    engine.runAndWait()
                    engine.stop()

            # Display
                if name != "":
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(img, name, (face_locations[3] + 6, face_locations[2]- 6), font, 0.5, (255, 255, 255), 1)
            cv2.imshow('img', img)
            cv2.waitKey(1)

    except KeyboardInterrupt:
        print("done!")
        cap.release()
        cv2.destroyAllWindows()


### Dispalys the video frames into Tkinter and also records new encodings in BG

In [8]:

def learn(video_src,ids,csv_source):
    global emp
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=HARISH;'
                      'Database=empDB;'
                      'Trusted_Connection=yes;')
    emp=pd.read_sql_query("select empname from empinfo where empid='%s'"% ids,conn)
    emp=emp.values.item()
    thread = threading.Thread(target=video2encoding, args=(0,eid))
    thread.start()
    cam.release() 
    root.update()

### This function calls add_new_encoding where encodings are saved in df and then it exports it into csv here.

In [9]:
def export():
    csv_source="D:\\P1\\encoding.csv"
    data_new = add_new_encoding(csv_source,faces,emp)
    os.chdir("D:\\P1")
    data_new.to_csv("encoding.csv",header=True,index=False)
    return(data_new)

## FRONT END (UI) - TKINTER

### Function contains the labels to be displayed in tkinter with its coordinates

In [10]:
def add_url():
    global eid
    global url_entry
    eid=url_entry.get()
    cnxn=pyodbc.connect('Driver={SQL Server};'
                      'Server=HARISH;'
                      'Database=empDB;'
                      'Trusted_Connection=yes;')
    cursor=cnxn.cursor()
    val=pd.read_sql_query("select empname,dob,gender,dept from empinfo where empid='%s'"% eid,cnxn)
    val=val.values
    ename=val[0][0]
    dob=val[0][1]
    gender=val[0][2]
    dept=val[0][3]
    head=['NAME:','DOB:','GENDER:','DEPARTMENT NAME:']
    enamel=StringVar()
    enamel.set(head[0])
    label1=Label(root,textvariable=enamel,justify=LEFT,font=('Helvetica', 11, 'bold'))
    label1.place(x =40,y =180)
    enamep=StringVar()
    enamep.set(ename.upper())
    label5=Label(root,textvariable=enamep,justify=LEFT,font=('Helvetica', 11),bg="white")
    label5.place(x =40,y =205)
    
    dobl=StringVar()
    dobl.set(head[1])
    label2=Label(root,textvariable=dobl,justify=LEFT,font=('Helvetica', 11, 'bold'))
    label2.place(x =40,y =240)
    dobp=StringVar()
    dobp.set(dob)
    label6=Label(root,textvariable=dobp,justify=LEFT,font=('Helvetica', 11),bg="white")
    label6.place(x =40,y =265)
    
    genderl=StringVar()
    genderl.set(head[2])
    label3=Label(root,textvariable=genderl,justify=LEFT,font=('Helvetica', 11, 'bold'))
    label3.place(x =40,y =300)
    genderp=StringVar()
    genderp.set(gender.upper())
    label7=Label(root,textvariable=genderp,justify=LEFT,font=('Helvetica', 11),bg="white")
    label7.place(x =40,y =325)
    
    deptl=StringVar()
    deptl.set(head[3])
    label4=Label(root,textvariable=deptl,justify=LEFT,font=('Helvetica', 11, 'bold'))
    label4.place(x =40,y =360)
    deptp=StringVar()
    deptp.set(dept.upper())
    label8=Label(root,textvariable=deptp,justify=LEFT,font=('Helvetica', 11),bg="white")
    label8.place(x =40,y =385)

### This function is called when CAPTURE button is clicked, which intiates the video encodings

In [11]:
def lena():
    learn(0,eid,"D:\\P1\\encoding.csv")

In [12]:
root =Tk()
var=StringVar()
label=Label(root,textvariable=var,font='Arial')
label.place(x =100,y =20)
#iurl=Button(root,text='SEARCH',font=('Helvetica', 14, 'bold'),compound = LEFT)
#iaddurl = PhotoImage(file ="C:\\Users\\harish\\Desktop\\Button\\final\\url.png")
#iurl= iaddurl.subsample(4,4)
root.geometry('1000x800')
label0=Label(root,text='EMPLOYEE ID',justify=LEFT,font=('Helvetica', 11, 'bold'))
label0.place(x =40,y =100)
url_entry = Entry(root,width = 25,borderwidth =7)
button_add = Button(root,text='SEARCH',font=('Helvetica', 14, 'bold'),compound = LEFT,command = add_url)
url_entry.place(x=40,y=130)
button_add.place(x=215,y=120)
capt = Button(root,text='CAPTURE',font=('Helvetica', 15, 'bold'),compound = LEFT,command=lena)
capt.place(x=160,y=500)
root.bind('<Escape>', lambda e: root.quit())
lmain = Label(root)
lmain.place(x=100,y=200)
root.mainloop()